## Easy

Calculate on paper O_1, O_2, O_3 after forward calculation in Vanilla Recurrent Neural Network

![https://s3-us-west-2.amazonaws.com/secure.notion-static.com/01b4e0a0-ca5a-47fd-932a-3880bbcbf1ae/Untitled_(1).jpg](https://s3-us-west-2.amazonaws.com/secure.notion-static.com/01b4e0a0-ca5a-47fd-932a-3880bbcbf1ae/Untitled_(1).jpg)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
W0 = np.array([0.2,0.3])
W1 = np.array([0.1,0.4])
b = 0.8
H = np.array([1,0.5])
X = np.array([[4,1],[2,0],[3,2]])

In [ ]:
for i in range(3):
  H = np.tanh(W0*X[i]+W1*H+b)
  print(f'O_{i} = {H}')

O_0 = [0.93540907 0.86172316]
O_1 = [0.86005109 0.81598709]
O_2 = [0.90258701 0.93862845]


## Medium

[IMDB Dataset of 50K Movie Reviews](https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews)

IMDB Dataset with movie reviews and their sentiment

Use  Keras' (or Pytorch) RNN, LSTM, GRU to classify IMDB dataset, and compare their accuracies

In [68]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, LSTM, GRU, SimpleRNN
from tensorflow.python.keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.layers import Dropout
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import load_model
from sklearn.model_selection import train_test_split
import re
import nltk
nltk.download("stopwords")
nltk.download("punkt")
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [52]:
data = pd.read_csv('./drive/MyDrive/NLP/IMDB Dataset.csv',error_bad_lines=False, engine="python",encoding='latin1')

In [53]:
WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('english')
#stop_word_list

In [54]:
data.sentiment = [ 1 if each == "positive" else 0 for each in data.sentiment]
sentiment = data['sentiment'].values

In [55]:
data['review'] = data['review'].apply(lambda x: re.sub('[,\.!?:()"]', '', x))
data['review'] = data['review'].apply(lambda x: re.sub('[^a-zA-Z"]', ' ', x))

data['review'] = data['review'].apply(lambda x: x.lower())

data['review'] = data['review'].apply(lambda x: x.strip())
data = data['review']

In [56]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data,sentiment,test_size = 0.2, random_state = 42)

In [57]:
from tensorflow.python.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 15000)
tokenizer.fit_on_texts(data)

In [58]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [59]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [60]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)

In [65]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [61]:
model = Sequential()

embedding_size = 50

model.add(Embedding(input_dim=15000,output_dim=embedding_size,input_length=max_tokens,name='embedding_layer'))

model.add(LSTM(units=16, return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(units=8, return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(units=4))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [66]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 567, 50)           750000    
_________________________________________________________________
lstm_3 (LSTM)                (None, 567, 16)           4288      
_________________________________________________________________
dropout_3 (Dropout)          (None, 567, 16)           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 567, 8)            800       
_________________________________________________________________
dropout_4 (Dropout)          (None, 567, 8)            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 4)                 208       
_________________________________________________________________
dropout_5 (Dropout)          (None, 4)                

In [67]:
history = model.fit(x_train_pad, y_train, validation_split=0.3, epochs=5, batch_size=1000, shuffle=True, verbose = 1)

Epoch 1/5
28/28 [==============================] - 104s 4s/step - loss: 0.6911 - accuracy: 0.5739 - val_loss: 0.6718 - val_accuracy: 0.6834
Epoch 2/5
28/28 [==============================] - 98s 3s/step - loss: 0.6376 - accuracy: 0.7145 - val_loss: 0.5236 - val_accuracy: 0.7682
Epoch 3/5
28/28 [==============================] - 98s 4s/step - loss: 0.4576 - accuracy: 0.8271 - val_loss: 0.4402 - val_accuracy: 0.8291
Epoch 4/5
28/28 [==============================] - 98s 4s/step - loss: 0.3723 - accuracy: 0.8795 - val_loss: 0.3916 - val_accuracy: 0.8519
Epoch 5/5
28/28 [==============================] - 98s 4s/step - loss: 0.3094 - accuracy: 0.9106 - val_loss: 0.3802 - val_accuracy: 0.8566


In [69]:
result = model.evaluate(x_test_pad, y_test)

313/313 [==============================] - 48s 152ms/step - loss: 0.3680 - accuracy: 0.8632


In [70]:
model = Sequential()

embedding_size = 50

model.add(Embedding(input_dim=15000,output_dim=embedding_size,input_length=max_tokens,name='embedding_layer'))

model.add(GRU(units=16, return_sequences=True))
model.add(Dropout(0.1))

model.add(GRU(units=8, return_sequences=True))
model.add(Dropout(0.1))

model.add(GRU(units=4))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [71]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 567, 50)           750000    
_________________________________________________________________
gru (GRU)                    (None, 567, 16)           3216      
_________________________________________________________________
dropout_6 (Dropout)          (None, 567, 16)           0         
_________________________________________________________________
gru_1 (GRU)                  (None, 567, 8)            600       
_________________________________________________________________
dropout_7 (Dropout)          (None, 567, 8)            0         
_________________________________________________________________
gru_2 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dropout_8 (Dropout)          (None, 4)                

In [72]:
history = model.fit(x_train_pad, y_train, validation_split=0.3, epochs=5, batch_size=1000, shuffle=True, verbose = 1)

Epoch 1/5
28/28 [==============================] - 87s 3s/step - loss: 0.6904 - accuracy: 0.5537 - val_loss: 0.6692 - val_accuracy: 0.6622
Epoch 2/5
28/28 [==============================] - 82s 3s/step - loss: 0.6405 - accuracy: 0.6957 - val_loss: 0.5679 - val_accuracy: 0.7088
Epoch 3/5
28/28 [==============================] - 80s 3s/step - loss: 0.5011 - accuracy: 0.7658 - val_loss: 0.4441 - val_accuracy: 0.8052
Epoch 4/5
28/28 [==============================] - 80s 3s/step - loss: 0.3414 - accuracy: 0.8687 - val_loss: 0.3607 - val_accuracy: 0.8549
Epoch 5/5
28/28 [==============================] - 81s 3s/step - loss: 0.2473 - accuracy: 0.9208 - val_loss: 0.3631 - val_accuracy: 0.8547


In [73]:
result = model.evaluate(x_test_pad, y_test)

313/313 [==============================] - 46s 143ms/step - loss: 0.3446 - accuracy: 0.8649


In [76]:
model = Sequential()

embedding_size = 50

model.add(Embedding(input_dim=15000,output_dim=embedding_size,input_length=max_tokens,name='embedding_layer'))

model.add(SimpleRNN(units=16, return_sequences=True))
model.add(Dropout(0.1))

model.add(SimpleRNN(units=8, return_sequences=True))
model.add(Dropout(0.1))

model.add(SimpleRNN(units=4))
model.add(Dropout(0.1))

model.add(Dense(1, activation='sigmoid'))

optimizer = Adam(lr=1e-3)

model.compile(loss='binary_crossentropy',optimizer=optimizer,metrics=['accuracy'])

In [77]:
history = model.fit(x_train_pad, y_train, validation_split=0.3, epochs=5, batch_size=1000, shuffle=True, verbose = 1)

Epoch 1/5
28/28 [==============================] - 36s 1s/step - loss: 0.7000 - accuracy: 0.5537 - val_loss: 0.5807 - val_accuracy: 0.7228
Epoch 2/5
28/28 [==============================] - 34s 1s/step - loss: 0.5318 - accuracy: 0.7669 - val_loss: 0.4569 - val_accuracy: 0.8134
Epoch 3/5
28/28 [==============================] - 34s 1s/step - loss: 0.4012 - accuracy: 0.8581 - val_loss: 0.4013 - val_accuracy: 0.8391
Epoch 4/5
28/28 [==============================] - 34s 1s/step - loss: 0.3288 - accuracy: 0.8911 - val_loss: 0.3855 - val_accuracy: 0.8492
Epoch 5/5
28/28 [==============================] - 34s 1s/step - loss: 0.2664 - accuracy: 0.9229 - val_loss: 0.3662 - val_accuracy: 0.8604


In [78]:
result = model.evaluate(x_test_pad, y_test)

313/313 [==============================] - 17s 53ms/step - loss: 0.3563 - accuracy: 0.8656


## Hard

[How to Develop Word-Based Neural Language Models in Python with Keras - Machine Learning Mastery](https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/)

Use neural networks from this approach to generate new names from our Grant names dataset

In [136]:
from tensorflow.keras.utils import to_categorical

In [125]:
df = pd.concat(pd.read_excel('./drive/MyDrive/NLP/grants.xlsx', sheet_name=[0,1,2,3,4]), ignore_index=True)[:-3]

In [126]:
df.tail()

,IKT,FIO,Points,University,No
4168,42300040.0,НҰРЛАН ЖАЙНА НҰРЛАНҚЫЗЫ,25.0,42.0,112.0
4169,38300434.0,ТӨКЕН ЖАРҚЫНАЙ ДЮСЕТҚЫЗЫ,25.0,38.0,113.0
4170,29300023.0,ЮЛДАШЕВ АЗИЗ МАСУТОВИЧ,25.0,29.0,114.0
4171,2300146.0,АҚЫЛБАЙ ҚҰРМЕТ ҚАНАТҰЛЫ,25.0,2.0,115.0
4172,48300001.0,МУХАМЕДИЕВ ТИМУР МАРАТОВИЧ,25.0,69.0,116.0


In [127]:
df['FIO'] = df['FIO'].apply(lambda fio: fio.lower())
df['FIO']

0                рахметулла асанәлі
1            жамашев ержан жеңісұлы
2        заманбек мухаммед жеңісұлы
3          жұмажанов мирас маратұлы
4       тлеулесов максут кайратович
                   ...             
4168        нұрлан жайна нұрланқызы
4169       төкен жарқынай дюсетқызы
4170         юлдашев азиз масутович
4171        ақылбай құрмет қанатұлы
4172     мухамедиев тимур маратович
Name: FIO, Length: 4173, dtype: object

In [128]:
data = '\n'.join(df['FIO'].values)
data 

'рахметулла асанәлі\nжамашев ержан жеңісұлы\nзаманбек мухаммед жеңісұлы\nжұмажанов мирас маратұлы\nтлеулесов максут кайратович\nдүйсенбек нұрдәулет жақсылықұлы\nтөлепберген ғалымжан алтынбайұлы\nсатыпалдиев бауыржан нұрсұлтанұлы\nшинкарова елена юрьевна\nдосмаханбет ербосын жанқозыбиұлы\nбегім нұржан ғалымұлы\nсапарқызы айзада\nәлімхан дінмұхамед бақытжанұлы\nикрам аружан алмасқызы\nбиғалиев ерназар дарханұлы\nалимсеитов олжас\nізтілеу саят бейбітұлы\nжетпісбай мақсат пердалыұлы\nбекен жалгас ержанулы\nермұхамбетова айдана алмасқызы\nмырзабеков фархат рашитұлы\nшадияр жансерік жораұлы\nбайбугаметова улжан викторовна\nсадвакасов мади курмангазыулы\nрамазан асылхан рамазанұлы\nқанапиянов нұржан ермекұлы\nжақсылық әйгерім бақытжанқызы\nамангелдинов әлем ернұрұлы\nжумадил кенжебай палжанұлы\nболатбекова даяна болатбекқызы\nкудагельди сейітжаппар маратұлы\nомарбаева диана көпболсынқызы\nсапарханқызы аягөз\nқалықова аяулым серікқызы\nнуртазина эльдана талгатовна\nайса дильназ жолдықызы\nерше

In [129]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded = tokenizer.texts_to_sequences([data])[0]

In [130]:
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary Size: %d' % vocab_size)

Vocabulary Size: 6847


In [131]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded)):
	sequence = encoded[i-1:i+1]
	sequences.append(sequence)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 12266


In [133]:
# split into X and y elements
sequences = np.array(sequences)
X, y = sequences[:,0],sequences[:,1]

In [137]:
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)

In [138]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1, 10)             68470     
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                12200     
_________________________________________________________________
dense_6 (Dense)              (None, 6847)              349197    
Total params: 429,867
Trainable params: 429,867
Non-trainable params: 0
_________________________________________________________________
None


In [139]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model.fit(X, y, epochs=500, verbose=2)

Epoch 1/500
384/384 - 5s - loss: 8.8224 - accuracy: 0.0054
Epoch 2/500
384/384 - 3s - loss: 8.4493 - accuracy: 0.0051
Epoch 3/500
384/384 - 3s - loss: 8.1367 - accuracy: 0.0060
Epoch 4/500
384/384 - 3s - loss: 7.9911 - accuracy: 0.0061
Epoch 5/500
384/384 - 3s - loss: 7.9153 - accuracy: 0.0059
Epoch 6/500
384/384 - 3s - loss: 7.8593 - accuracy: 0.0057
Epoch 7/500
384/384 - 3s - loss: 7.8087 - accuracy: 0.0062
Epoch 8/500
384/384 - 3s - loss: 7.7583 - accuracy: 0.0062
Epoch 9/500
384/384 - 3s - loss: 7.6997 - accuracy: 0.0064
Epoch 10/500
384/384 - 3s - loss: 7.6135 - accuracy: 0.0075
Epoch 11/500
384/384 - 3s - loss: 7.4705 - accuracy: 0.0100
Epoch 12/500
384/384 - 3s - loss: 7.2478 - accuracy: 0.0131
Epoch 13/500
384/384 - 3s - loss: 6.9477 - accuracy: 0.0172
Epoch 14/500
384/384 - 3s - loss: 6.6145 - accuracy: 0.0298
Epoch 15/500
384/384 - 3s - loss: 6.2889 - accuracy: 0.0444
Epoch 16/500
384/384 - 3s - loss: 5.9737 - accuracy: 0.0628
Epoch 17/500
384/384 - 3s - loss: 5.6672 - accura

In [146]:
# evaluate
in_text = 'Че'
print(in_text)
encoded = tokenizer.texts_to_sequences([in_text])[0]
encoded = np.array(encoded)
yhat = model.predict_classes(encoded, verbose=0)
for word, index in tokenizer.word_index.items():
	if index == yhat:
		print(word)

Че
никита


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
